In [1]:
import time
import json
import sys
import os
import glob
import re
import logging
import traceback

from collections import deque

from globus_automate_client import (create_flows_client, graphviz_format, state_colors_for_log,
                                    create_action_client, 
                                    create_flows_client)
from funcx.sdk.client import FuncXClient
from funcx.serialize import FuncXSerializer


CLIENT_ID = "e6c75d97-532a-4c88-b031-8584a319fa3e"

In [2]:
fxc = FuncXClient()
# fxs = FuncXSerializer() # (de)serialize the funcx codes

In [3]:
def ptycho(data):
    """Test the ptycho tool"""
    import os
    import subprocess
    import logging
    from subprocess import PIPE
    
    rec_max_qsize = data['max_qsize']
    rec_ngpu = data['rec_ngpu']
    wid = data['wid']
    dataset_name = data['dataset_name']
    
    log_file_name = "/grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/logs/funcx-ptycho-{}-w{}-q{}-g{}.log".format(dataset_name, wid, rec_max_qsize, rec_ngpu)
    
    logging.basicConfig(filename=log_file_name,
                        filemode='a',
                        format='%(asctime)s %(levelname)s %(message)s',
                        level=logging.INFO,
                        datefmt='%Y-%m-%d %H:%M:%S')

    
    
    logging.info("Starting ptycho funcx function.")
    
    remote_log_file_name = "/grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/logs/tike-wf-{}-w{}-q{}-g{}.log".format(dataset_name, wid, rec_max_qsize, rec_ngpu)
    
    python_path = data['python_path']
    script_path = data['script_path']
    
    #recon. script parameters
    ifpath = data['ifpath']
    ofpath = data['ofpath']
    rec_alg = data['rec_alg']
    rec_nmodes = data['rec_nmodes']
    rec_niter = data['rec_niter']
    rec_output_freq = data['rec_output_freq']
    rec_recover_psi = '--recover-psi' if (('rec_recover_psi' in data) and data['rec_recover_psi']) else ''
    rec_recover_probe = '--recover-probe' if (('rec_recover_probe' in data) and data['rec_recover_probe']) else ''
    rec_recover_positions = '--recover-positions' if (('rec_recover_positions' in data) and data['rec_recover_positions']) else ''
    rec_model = data['rec_model']
    rec_use_mpi = '--use-mpi' if (('rec_use_mpi' in data) and data['rec_use_mpi']) else ''
    rec_overwrite = '--overwrite' if (('rec_overwrite' in data) and data['rec_overwrite']) else ''
    rec_auto_pin = '--auto-pin' if (('rec_auto_pin' in data) and data['rec_auto_pin']) else ''
    rec_gpu_id = data['rec_gpu_id']
    rec_log_filename = remote_log_file_name
    
    try:
        os.mkdir(ofpath)
    except:
        pass
    
    cmd = f"{python_path} {script_path} --algorithm={rec_alg} --nmodes={rec_nmodes} --niter={rec_niter} --output-freq={rec_output_freq} {rec_recover_psi} {rec_recover_probe} {rec_recover_positions} --model={rec_model} --ngpu={rec_ngpu} {rec_use_mpi} --ifile='{ifpath}' {rec_overwrite} {rec_auto_pin} --gpu-id={rec_gpu_id} --folder='{ofpath}' --wf-max-qsize={rec_max_qsize} --log-file='{rec_log_filename}'"
    logging.info(f"Running command: {cmd}")
    
    try:
        res = subprocess.run(cmd, stdout=PIPE, stderr=PIPE,
                             shell=True, executable='/bin/bash')
    except:
        pass
    outstr = f"{res.stdout}"
    return outstr
    
func_ptycho_uuid = fxc.register_function(ptycho)
print(func_ptycho_uuid)

9413b43a-51b6-46e2-b605-e95eab566250


In [4]:
def get_folder_paths(path):
    import glob
    import re
    
    return sorted(glob.glob(path, recursive=False),
                key = lambda v : int(re.search(r"(\d+)" , v[len(v)-"".join(reversed(v)).index('/'):]).group(0)))


def get_file_paths(path):
    import glob
    import re
    
    return sorted(glob.glob(path, recursive=False),
                key = lambda v : int(re.search(r"(\d+)" , v[len(v)-"".join((reversed(v))).index('/'):v.index('.')]).group(0)))

fx_func_get_file_paths_uuid = fxc.register_function(get_file_paths)
fx_func_get_folder_paths_uuid = fxc.register_function(get_folder_paths)
print(fx_func_get_file_paths_uuid)
print(fx_func_get_folder_paths_uuid)

3d0190a2-0307-4ed7-97d3-8d68ef6c9e27
2b1c058b-eba7-49db-8478-1fe26c6fc006


In [5]:
flow_definition = {
  "Comment": "An analysis flow",
  "StartAt": "Transfer",
  "States": {
    "Transfer": {
      "Comment": "Initial transfer",
      "Type": "Action",
      "ActionUrl": "https://actions.automate.globus.org/transfer/transfer",
      "ActionScope": "https://auth.globus.org/scopes/actions.globus.org/transfer/transfer",
      "Parameters": {
        "source_endpoint_id.$": "$.input.source_endpoint", 
        "destination_endpoint_id.$": "$.input.dest_endpoint",
        "transfer_items": [
          {
            "source_path.$": "$.input.source_path",
            "destination_path.$": "$.input.dest_path",
            "recursive": True
          }
        ]
      },
      "ResultPath": "$.Transfer1Result",
      "WaitTime": 14400,
      "Next": "Analyze"
    },
    "Analyze": {
      "Comment": "Run a funcX function",
      "Type": "Action",
      "ActionUrl": "https://api.funcx.org/automate",
      "ActionScope": "https://auth.globus.org/scopes/facd7ccc-c5f4-42aa-916b-a0e270e2c2a9/automate2",
      "Parameters": {
          "tasks": [{
            "endpoint.$": "$.input.fx_ep",
            "func.$": "$.input.fx_id",
            "payload.$": "$.input.params"
        }]
      },
      "ResultPath": "$.AnalyzeResult",
      "WaitTime": 14400,
      "Next": "Transfer2"
    },
    "Transfer2": {
      "Comment": "Return transfer",
      "Type": "Action",
      "ActionUrl": "https://actions.automate.globus.org/transfer/transfer",
      "ActionScope": "https://auth.globus.org/scopes/actions.globus.org/transfer/transfer",
      "Parameters": {
        "source_endpoint_id.$": "$.input.dest_endpoint", 
        "destination_endpoint_id.$": "$.input.source_endpoint",
        "transfer_items": [
          {
            "source_path.$": "$.input.result_path",
            "destination_path.$": "$.input.source_result_path",
            "recursive": True #False
          }
        ]
      },
      "ResultPath": "$.Transfer2Result",
      "WaitTime": 14400,
      "End": True
    },
  }
}

In [6]:
prisma_fx_endpoint = '4bf59543-3398-42d2-9416-c628e9f5635f'

src_wf_root_path = '/prisma-data1/bicer/workflow'
src_input_folder_prefix = "input"
src_output_folder_prefix = "output"

dest_wf_root_path = '/grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate'
dest_input_folder_prefix = "input"
dest_output_folder_prefix = "output"


src_input_folder_paths_regex = f"{src_wf_root_path}/{src_input_folder_prefix}/*"
rid = fxc.run(src_input_folder_paths_regex, 
              endpoint_id=prisma_fx_endpoint, 
              function_id=fx_func_get_folder_paths_uuid)
src_input_folder_paths = fxc.get_result(rid)

src_output_folder_paths = []
dest_output_folder_paths = []
dest_input_folder_paths = []
for src_input_folder_path in src_input_folder_paths:
    #print(src_input_folder_path)
    iid = re.findall(r'\d+', src_input_folder_path)
    src_output_folder_path = f"{src_wf_root_path}/{src_output_folder_prefix}/{iid[-1]}"
    src_output_folder_paths.append(src_output_folder_path)
    dest_input_folder_path = f"{dest_wf_root_path}/{dest_input_folder_prefix}/{iid[-1]}"
    dest_input_folder_paths.append(dest_input_folder_path)
    dest_output_folder_path = f"{dest_wf_root_path}/{dest_output_folder_prefix}/{iid[-1]}"
    dest_output_folder_paths.append(dest_output_folder_path)
    
    

# src_input_folder_paths: diffraction patterh files to be processed @ APS
# src_output_folder_paths: folders for reconstrcuted images after processing @ APS
# dest_input_folder_paths: diffraction patterh files to be processed @ ALCF
# dest_output_folder_paths: folders for reconstrcuted images after processing @ ALCF

for (src_input_folder_path, src_output_folder_path, dest_input_folder_path, dest_output_folder_path ) in zip(src_input_folder_paths, src_output_folder_paths, dest_input_folder_paths, dest_output_folder_paths):
    print(f"Source input folder: {src_input_folder_path}; Source output folder: {src_output_folder_path}")
    print(f"Dest. input folder: {dest_input_folder_path}; Dest. output folder: {dest_output_folder_path}")
    print()

Source input folder: /prisma-data1/bicer/workflow/input/scan192; Source output folder: /prisma-data1/bicer/workflow/output/192
Dest. input folder: /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/input/192; Dest. output folder: /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/output/192

Source input folder: /prisma-data1/bicer/workflow/input/scan193; Source output folder: /prisma-data1/bicer/workflow/output/193
Dest. input folder: /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/input/193; Dest. output folder: /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/output/193

Source input folder: /prisma-data1/bicer/workflow/input/scan194; Source output folder: /prisma-data1/bicer/workflow/output/194
Dest. input folder: /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/input/194; Dest. output folder: /grand/hp-ptycho/bicer/ptycho/comm_33IDD/globus_automate/output/194

Source input folder: /prisma-data1/bicer/workflow/input/scan195; Source output folder

In [7]:
theta_fx_endpoint = '2ab22e1f-4cf1-47e9-a40c-dd8c58b41d73' #theta-ptycho-8w-1n

#src_endpoint = '9c9cb97e-de86-11e6-9d15-22000a1e3b52' #'aps#data' # Voyager
src_endpoint = 'dd916908-0072-11e7-badc-22000b9a448b' #'hostel' #aps/workstation
#dest_endpoint = '08925f04-569f-11e7-bef8-22000b9a448b' #'alcf#dtn_theta' # Theta DTN # 'e09e65f5-6d04-11e5-ba46-22000b92c6ec' #
dest_endpoint = '08925f04-569f-11e7-bef8-22000b9a448b' #'e09e65f5-6d04-11e5-ba46-22000b92c6ec' #'alcf#dtn_mira'

# Ptycho recon params
script_path = '/home/bicer/projects/tike/scripts/tike-pinned-ptycho-recon.py'
python_path = "/home/bicer/projects/tyler/bin/python"

rec_alg = 'cgrad'
rec_nmodes = 1
rec_upd_pos = False
rec_niter = 200
rec_output_freq = 200
rec_recover_psi = True
rec_recover_probe= True
rec_recover_positions = False
rec_model = 'gaussian'
rec_ngpu = 1
rec_use_mpi = False
rec_overwrite = True
rec_auto_pin = True


dataset_name = "catalyst"
nnodes = 8
nworkers_per_node = 8
MAX_QSIZE = nnodes * nworkers_per_node


flow_inputs = []
gcounter = 0
for (src_input_folder_path, src_output_folder_path, 
     dest_input_folder_path, dest_output_folder_path ) in zip(
    src_input_folder_paths, src_output_folder_paths, 
    dest_input_folder_paths, dest_output_folder_paths):
    
    rec_gpu_id = gcounter%nworkers_per_node
    
    flow_input = {
        "input": {
            "source_endpoint": f"{src_endpoint}",
            "source_path": f"{src_input_folder_path}",
            "dest_endpoint": dest_endpoint,
            "dest_path": f"{dest_input_folder_path}",

            "result_path": f"{dest_output_folder_path}",#f"{dest_resultpath}/out_{src_filename}",
            "source_result_path": f"{src_output_folder_path}", #/out_{src_filename}",
            "fx_ep": f"{theta_fx_endpoint}",
            "fx_id": f"{func_ptycho_uuid}",
            "params": {'ifpath': f"{dest_input_folder_path}",
                       'ofpath': f"{dest_output_folder_path}/",
                       'script_path': script_path,
                       'python_path': python_path,
                       'rec_alg': rec_alg,
                       'rec_nmodes': rec_nmodes,
                       'rec_upd_pos': rec_upd_pos,
                       'rec_niter': rec_niter,
                       'rec_output_freq': rec_output_freq,
                       'rec_recover_psi': rec_recover_psi,
                       'rec_recover_probe': rec_recover_probe,
                       'rec_recover_positions': rec_recover_positions,
                       'rec_model': rec_model,
                       'rec_ngpu': rec_ngpu,
                       'rec_use_mpi': rec_use_mpi,
                       'rec_overwrite': rec_overwrite,
                       'rec_auto_pin': rec_auto_pin,
                       'rec_gpu_id':rec_gpu_id,
                       'max_qsize': MAX_QSIZE,
                       'dataset_name': dataset_name,
                       'wid':gcounter}
        }
    }
    gcounter=gcounter+1
    flow_inputs.append(flow_input)

#print(f"transfer file from {src_endpoint}#{src_filepath}/{src_filename} to {dest_endpoint}#{dest_filepath}/")
#print(f"recon file:{dest_filepath}/{src_filename} output:{dest_resultpath}/")
#print(f"transfer file from {dest_endpoint}:{dest_resultpath} to {src_endpoint}#{src_result_path}/")

In [8]:
print(len(flow_inputs))

168


In [9]:
flows_client = create_flows_client(CLIENT_ID)
flow = flows_client.deploy_flow(flow_definition, title="Simple ptycho data analysis flow")
flow_id = flow['id']
flow_scope = flow['globus_auth_scope']

print(flow)
#print(f'Newly created flow with id:\n{flow_id}\nand scope:\n{flow_scope}')

GlobusHTTPResponse({'action_url': 'https://flows.globus.org/flows/f09e8a0f-350a-4642-8fa5-477d364a97e3', 'administered_by': [], 'api_version': '1.0', 'created_at': '2021-06-03T12:29:03.587517+00:00', 'created_by': 'urn:globus:auth:identity:a478e9c0-d274-11e5-9901-63aaec352b80', 'definition': {'Comment': 'An analysis flow', 'StartAt': 'Transfer', 'States': {'Analyze': {'ActionScope': 'https://auth.globus.org/scopes/facd7ccc-c5f4-42aa-916b-a0e270e2c2a9/automate2', 'ActionUrl': 'https://api.funcx.org/automate', 'Comment': 'Run a funcX function', 'Next': 'Transfer2', 'Parameters': {'tasks': [{'endpoint.$': '$.input.fx_ep', 'func.$': '$.input.fx_id', 'payload.$': '$.input.params'}]}, 'ResultPath': '$.AnalyzeResult', 'Type': 'Action', 'WaitTime': 14400}, 'Transfer': {'ActionScope': 'https://auth.globus.org/scopes/actions.globus.org/transfer/transfer', 'ActionUrl': 'https://actions.automate.globus.org/transfer/transfer', 'Comment': 'Initial transfer', 'Next': 'Analyze', 'Parameters': {'destin

In [10]:
log_file_name = "/home/beams/TBICER/logs/funcx-ptycho-wf-{}-q{}-g{}.log".format(dataset_name, MAX_QSIZE, rec_ngpu)
logging.basicConfig(filename=log_file_name,
                    filemode='a',
                    format='%(asctime)s %(levelname)s %(message)s',
                    level=logging.INFO,
                    datefmt='%Y-%m-%d %H:%M:%S')

q0 = deque()
q1 = deque()

logging.info(f"Workflow information for: {log_file_name}")
logging.info(f"Deployed flow: {flow}")
nnodes = 8
nworkers_per_node = 8

counter=0
nflows = len(flow_inputs)
logging.info(f"Starting workflow {dataset_name}:{MAX_QSIZE}:{nnodes} left={nflows-counter} succeeded={counter}")

for i in range(len(flow_inputs)):
        flow_action = flows_client.run_flow(flow_id, flow_scope, flow_inputs[i])
        q1.append(flow_action)
        lstr=f"Flow {i} initiated and added to q1: {flow_action['action_id']}"
        logging.info(lstr)
        print(lstr)

i=-1
while len(q1)>0:
    i = (i+1)%len(q1)
    try:
        flow = flows_client.flow_action_status(flow_id, flow_scope, q1[i]['action_id'])
    except Exception as exc:
        err = traceback.format_exc()
        print(f"Exception:{exc}")
        logging.warning(err)
        continue
        
    
    lstr = f"len(q0)={len(q0)}; len(q1)={len(q1)}; i={i}"
    logging.info(lstr)
    print(lstr)
    
    lstr = f"Flow {i} status: {q1[i]['action_id']}: {flow['status']}" 
    logging.info(lstr)
    print(lstr)
    
    if len(q1) < 4: time.sleep(1)
    
    if flow['status'] == 'SUCCEEDED':
        logging.info(f"Flow succeeded: {q1[i]}")
        del q1[i]
        counter = counter+1
        if len(q0)>0:
            flow_input = q0.popleft()
            flow_action = flows_client.run_flow(flow_id, flow_scope, flow_input)
            q1.append(flow_action)
            lstr = f"New flow initiated and added to the q1: {flow_action['action_id']}"
            logging.info(lstr)
            print(lstr)
            lstr=f"Copy from {flow_input['input']['params']['ifpath']} to {flow_input['input']['params']['ofpath']}"
            logging.info(lstr)
            print(lstr)
            
    if flow['status'] == 'FAILED':
        lstr = f"Failed action, removing: {flow_action['action_id']}"
        print(lstr)
        logging.warning(lstr)
        
        del q1[i]
        
logging.info(f"Done workflow {dataset_name}:{MAX_QSIZE}:{nnodes} left={nflows-counter} succeeded={counter}")

Starting login with Globus Auth, press ^C to cancel.
Flow 0 initiated and added to q1: 820547a9-6bbb-4842-8350-b83f63f67e5f
Flow 1 initiated and added to q1: 05c1d5f0-864c-4dec-9e0d-7f1fd40d060b
Flow 2 initiated and added to q1: af85f879-595a-475b-ab52-d2b98b9dd120
Flow 3 initiated and added to q1: 76d18ea6-5005-4e62-a1b8-3ae1bb618948
Flow 4 initiated and added to q1: 6880d934-d288-4645-91bf-3ccaba8a3af7
Flow 5 initiated and added to q1: c0184256-7017-446f-9bb9-30fd6ecb0775
Flow 6 initiated and added to q1: 4a989727-5417-4c42-97f8-9c765028a8ed
Flow 7 initiated and added to q1: 34810f4b-d27b-40ba-b25e-e3931bf7da71
Flow 8 initiated and added to q1: e0899206-8120-4059-8ed3-34e576084f1f
Flow 9 initiated and added to q1: e4a1a591-1147-435d-b6ca-7f118a9530f2
Flow 10 initiated and added to q1: a44fbd11-c1d0-47e2-84ed-c87155cc1029
Flow 11 initiated and added to q1: 634dfc34-007d-4514-bdc7-ebf2333ffac3
Flow 12 initiated and added to q1: 4d961426-f84e-434c-b0e4-4b3d05b64d42
Flow 13 initiated and

Flow 114 initiated and added to q1: b68ee6e6-a11c-46c2-a456-bf14ae114a3f
Flow 115 initiated and added to q1: f859b45b-276f-450c-9985-260a4fe7fb3e
Flow 116 initiated and added to q1: 13deb6f5-be32-435a-aa47-3a53de5ce27f
Flow 117 initiated and added to q1: 8b15f968-6972-476e-8221-6797d8315889
Flow 118 initiated and added to q1: 237e877c-5ae9-441b-a3e9-e8ac9f5a719a
Flow 119 initiated and added to q1: 799092d6-5279-4246-a0e9-81e990ac2ae5
Flow 120 initiated and added to q1: 0307faab-6f69-483b-b36c-abdbe301e197
Flow 121 initiated and added to q1: 7482baa5-af0b-4e40-b5e0-44f20b059c21
Flow 122 initiated and added to q1: 269084af-144a-4d60-b4c9-62e19396fe03
Flow 123 initiated and added to q1: 6766e3be-9e01-4faa-b815-78e6a5230638
Flow 124 initiated and added to q1: 7cbe2cc0-bea5-4b1b-aa09-ecb9564f95d9
Flow 125 initiated and added to q1: b12e108f-cd2e-41d4-9d5d-e12dabbadf87
Flow 126 initiated and added to q1: 747d49b1-c6c4-4e27-aeca-34f5221204a7
Flow 127 initiated and added to q1: 145236a4-2ca8-4

len(q0)=0; len(q1)=162; i=48
Flow 48 status: d6de149a-226d-4609-a044-f65311d123c1: ACTIVE
len(q0)=0; len(q1)=162; i=49
Flow 49 status: 1e9bd8f0-17e0-4821-8c0b-3be5e249d845: ACTIVE
len(q0)=0; len(q1)=162; i=50
Flow 50 status: e736024b-1c87-4a16-b471-a87af4f22cb2: ACTIVE
len(q0)=0; len(q1)=162; i=51
Flow 51 status: a17ab796-8e40-40cd-b328-467f62578767: ACTIVE
len(q0)=0; len(q1)=162; i=52
Flow 52 status: 932c2e8e-2852-4683-88e4-2b52f36f5a48: ACTIVE
len(q0)=0; len(q1)=162; i=53
Flow 53 status: a58bc35f-389e-4ea3-a952-a22b5fa0ab36: ACTIVE
len(q0)=0; len(q1)=162; i=54
Flow 54 status: 454cfc9b-bb3d-4e93-8e32-08984cf2b599: ACTIVE
len(q0)=0; len(q1)=162; i=55
Flow 55 status: 275b2afa-2140-406c-9508-84d716727306: ACTIVE
len(q0)=0; len(q1)=162; i=56
Flow 56 status: d9350d54-c6b1-4ccf-bf4a-346d3a16d635: ACTIVE
len(q0)=0; len(q1)=162; i=57
Flow 57 status: 620668ce-4b28-4ec6-b917-de30c2bd7b61: ACTIVE
len(q0)=0; len(q1)=162; i=58
Flow 58 status: b358e874-d0ec-4a8b-b41a-5ca38705da30: ACTIVE
len(q0)=0;

len(q0)=0; len(q1)=162; i=139
Flow 139 status: 07804291-e78e-4a48-a755-53eb2d4f199c: ACTIVE
len(q0)=0; len(q1)=162; i=140
Flow 140 status: adbd5777-a9a8-4b95-a03a-dd2b9a7538be: ACTIVE
len(q0)=0; len(q1)=162; i=141
Flow 141 status: 231abe78-f1f0-44c7-b0b4-4006b29e5fa7: ACTIVE
len(q0)=0; len(q1)=162; i=142
Flow 142 status: 19ee397b-ba78-4259-bf48-11bba09f04e7: ACTIVE
len(q0)=0; len(q1)=162; i=143
Flow 143 status: 8e6171ad-2de7-48c8-ad5e-7c2dde8e983d: ACTIVE
len(q0)=0; len(q1)=162; i=144
Flow 144 status: 1b5884c2-9fe1-49cb-9171-3f72b0b21405: ACTIVE
len(q0)=0; len(q1)=162; i=145
Flow 145 status: 75a4bd6f-23a9-4ea0-a534-f6ab74e1556a: ACTIVE
len(q0)=0; len(q1)=162; i=146
Flow 146 status: 2877aff6-2557-4f14-a08d-f30ff4de4ceb: ACTIVE
len(q0)=0; len(q1)=162; i=147
Flow 147 status: 4ca409f5-e290-429e-848d-2b846353bed8: ACTIVE
len(q0)=0; len(q1)=162; i=148
Flow 148 status: da2b03c3-18ab-4a1b-97aa-14bd3b12ee6c: ACTIVE
len(q0)=0; len(q1)=162; i=149
Flow 149 status: e54363fa-10c3-4ba6-8623-9711ff39e

len(q0)=0; len(q1)=159; i=68
Flow 68 status: 8b0bddc2-8afe-4c7a-ad2c-ca848e2c220a: ACTIVE
len(q0)=0; len(q1)=159; i=69
Flow 69 status: 308ccb26-0cb4-4d23-bf1d-cd10e5e282fa: ACTIVE
len(q0)=0; len(q1)=159; i=70
Flow 70 status: fe71913f-19b5-43bd-b6a9-a5ffe557a467: ACTIVE
len(q0)=0; len(q1)=159; i=71
Flow 71 status: 7f768791-79aa-4011-a626-9cfd1deb6991: ACTIVE
len(q0)=0; len(q1)=159; i=72
Flow 72 status: 2fd293ff-1893-4ab3-b817-4363414dc319: ACTIVE
len(q0)=0; len(q1)=159; i=73
Flow 73 status: e10a6f52-63d8-4b56-8c3f-0e3b78bb2d11: ACTIVE
len(q0)=0; len(q1)=159; i=74
Flow 74 status: e65c6a99-814a-4e6f-8132-09b37578edbd: ACTIVE
len(q0)=0; len(q1)=159; i=75
Flow 75 status: 7dbdd890-2e3b-4302-bc0a-53a6afefb84c: ACTIVE
len(q0)=0; len(q1)=159; i=76
Flow 76 status: 0ee8c2e1-6963-4a78-82ce-401b3183e4f4: ACTIVE
len(q0)=0; len(q1)=159; i=77
Flow 77 status: d0133782-575f-42fe-a988-74b5c39d21d4: ACTIVE
len(q0)=0; len(q1)=159; i=78
Flow 78 status: 9282bc10-12de-4c18-84c8-6c644ac0d0db: ACTIVE
len(q0)=0;

...

len(q0)=0; len(q1)=24; i=19
Flow 19 status: 62d45372-67f3-4f51-8c01-1a0abac5a4d9: ACTIVE
len(q0)=0; len(q1)=24; i=20
Flow 20 status: 5545669d-54e0-41c6-b850-ca78213801e9: ACTIVE
len(q0)=0; len(q1)=24; i=21
Flow 21 status: 3c78dbdd-c7e4-4af9-a0b7-e8ac224f0bec: ACTIVE
len(q0)=0; len(q1)=24; i=22
Flow 22 status: 967a4b0e-60c0-49db-ae18-086b1cd940d3: ACTIVE
len(q0)=0; len(q1)=24; i=23
Flow 23 status: 12b0aac5-ea12-4007-bdc0-b4b1402730e9: ACTIVE
len(q0)=0; len(q1)=24; i=0
Flow 0 status: 31c5819e-9786-44a9-a2f2-21aaaa300e9c: ACTIVE
len(q0)=0; len(q1)=24; i=1
Flow 1 status: 07804291-e78e-4a48-a755-53eb2d4f199c: ACTIVE
len(q0)=0; len(q1)=24; i=2
Flow 2 status: adbd5777-a9a8-4b95-a03a-dd2b9a7538be: ACTIVE
len(q0)=0; len(q1)=24; i=3
Flow 3 status: 231abe78-f1f0-44c7-b0b4-4006b29e5fa7: ACTIVE
len(q0)=0; len(q1)=24; i=4
Flow 4 status: 19ee397b-ba78-4259-bf48-11bba09f04e7: ACTIVE
len(q0)=0; len(q1)=24; i=5
Flow 5 status: 8e6171ad-2de7-48c8-ad5e-7c2dde8e983d: ACTIVE
len(q0)=0; len(q1)=24; i=6
Flow 6

len(q0)=0; len(q1)=24; i=16
Flow 16 status: b8ffc6f2-f5a9-47e9-a586-5c8080f28d94: ACTIVE
len(q0)=0; len(q1)=24; i=17
Flow 17 status: 628ee5c2-6780-4fcc-84d1-3926fd4268bf: ACTIVE
len(q0)=0; len(q1)=24; i=18
Flow 18 status: 867f8eb4-c41f-443b-b17d-3dad9033f829: ACTIVE
len(q0)=0; len(q1)=24; i=19
Flow 19 status: 62d45372-67f3-4f51-8c01-1a0abac5a4d9: ACTIVE
len(q0)=0; len(q1)=24; i=20
Flow 20 status: 5545669d-54e0-41c6-b850-ca78213801e9: ACTIVE
len(q0)=0; len(q1)=24; i=21
Flow 21 status: 3c78dbdd-c7e4-4af9-a0b7-e8ac224f0bec: ACTIVE
len(q0)=0; len(q1)=24; i=22
Flow 22 status: 967a4b0e-60c0-49db-ae18-086b1cd940d3: ACTIVE
len(q0)=0; len(q1)=24; i=23
Flow 23 status: 12b0aac5-ea12-4007-bdc0-b4b1402730e9: ACTIVE
len(q0)=0; len(q1)=24; i=0
Flow 0 status: 31c5819e-9786-44a9-a2f2-21aaaa300e9c: ACTIVE
len(q0)=0; len(q1)=24; i=1
Flow 1 status: 07804291-e78e-4a48-a755-53eb2d4f199c: ACTIVE
len(q0)=0; len(q1)=24; i=2
Flow 2 status: adbd5777-a9a8-4b95-a03a-dd2b9a7538be: ACTIVE
len(q0)=0; len(q1)=24; i=3


len(q0)=0; len(q1)=24; i=13
Flow 13 status: 2d7e1fa0-f914-4cef-be2c-5efa74b6eaf9: ACTIVE
len(q0)=0; len(q1)=24; i=14
Flow 14 status: 5c36bae9-9675-48dc-b9a1-0594654f5608: ACTIVE
len(q0)=0; len(q1)=24; i=15
Flow 15 status: 483d437d-852c-4d9a-a1b7-113447435be5: ACTIVE
len(q0)=0; len(q1)=24; i=16
Flow 16 status: b8ffc6f2-f5a9-47e9-a586-5c8080f28d94: ACTIVE
len(q0)=0; len(q1)=24; i=17
Flow 17 status: 628ee5c2-6780-4fcc-84d1-3926fd4268bf: ACTIVE
len(q0)=0; len(q1)=24; i=18
Flow 18 status: 867f8eb4-c41f-443b-b17d-3dad9033f829: ACTIVE
len(q0)=0; len(q1)=24; i=19
Flow 19 status: 62d45372-67f3-4f51-8c01-1a0abac5a4d9: ACTIVE
len(q0)=0; len(q1)=24; i=20
Flow 20 status: 5545669d-54e0-41c6-b850-ca78213801e9: ACTIVE
len(q0)=0; len(q1)=24; i=21
Flow 21 status: 3c78dbdd-c7e4-4af9-a0b7-e8ac224f0bec: ACTIVE
len(q0)=0; len(q1)=24; i=22
Flow 22 status: 967a4b0e-60c0-49db-ae18-086b1cd940d3: ACTIVE
len(q0)=0; len(q1)=24; i=23
Flow 23 status: 12b0aac5-ea12-4007-bdc0-b4b1402730e9: ACTIVE
len(q0)=0; len(q1)=24

len(q0)=0; len(q1)=24; i=10
Flow 10 status: da2b03c3-18ab-4a1b-97aa-14bd3b12ee6c: ACTIVE
len(q0)=0; len(q1)=24; i=11
Flow 11 status: e54363fa-10c3-4ba6-8623-9711ff39e8c4: ACTIVE
len(q0)=0; len(q1)=24; i=12
Flow 12 status: 821df025-53c5-4e8b-b316-a1e5477f9de9: ACTIVE
len(q0)=0; len(q1)=24; i=13
Flow 13 status: 2d7e1fa0-f914-4cef-be2c-5efa74b6eaf9: ACTIVE
len(q0)=0; len(q1)=24; i=14
Flow 14 status: 5c36bae9-9675-48dc-b9a1-0594654f5608: ACTIVE
len(q0)=0; len(q1)=24; i=15
Flow 15 status: 483d437d-852c-4d9a-a1b7-113447435be5: ACTIVE
len(q0)=0; len(q1)=24; i=16
Flow 16 status: b8ffc6f2-f5a9-47e9-a586-5c8080f28d94: ACTIVE
len(q0)=0; len(q1)=24; i=17
Flow 17 status: 628ee5c2-6780-4fcc-84d1-3926fd4268bf: ACTIVE
len(q0)=0; len(q1)=24; i=18
Flow 18 status: 867f8eb4-c41f-443b-b17d-3dad9033f829: ACTIVE
len(q0)=0; len(q1)=24; i=19
Flow 19 status: 62d45372-67f3-4f51-8c01-1a0abac5a4d9: ACTIVE
len(q0)=0; len(q1)=24; i=20
Flow 20 status: 5545669d-54e0-41c6-b850-ca78213801e9: ACTIVE
len(q0)=0; len(q1)=24

len(q0)=0; len(q1)=17; i=9
Flow 9 status: b8ffc6f2-f5a9-47e9-a586-5c8080f28d94: ACTIVE
len(q0)=0; len(q1)=17; i=10
Flow 10 status: 628ee5c2-6780-4fcc-84d1-3926fd4268bf: INACTIVE
len(q0)=0; len(q1)=17; i=11
Flow 11 status: 867f8eb4-c41f-443b-b17d-3dad9033f829: ACTIVE
len(q0)=0; len(q1)=17; i=12
Flow 12 status: 62d45372-67f3-4f51-8c01-1a0abac5a4d9: ACTIVE
len(q0)=0; len(q1)=17; i=13
Flow 13 status: 5545669d-54e0-41c6-b850-ca78213801e9: ACTIVE
len(q0)=0; len(q1)=17; i=14
Flow 14 status: 3c78dbdd-c7e4-4af9-a0b7-e8ac224f0bec: ACTIVE
len(q0)=0; len(q1)=17; i=15
Flow 15 status: 967a4b0e-60c0-49db-ae18-086b1cd940d3: ACTIVE
len(q0)=0; len(q1)=17; i=16
Flow 16 status: 12b0aac5-ea12-4007-bdc0-b4b1402730e9: ACTIVE
len(q0)=0; len(q1)=17; i=0
Flow 0 status: adbd5777-a9a8-4b95-a03a-dd2b9a7538be: SUCCEEDED
len(q0)=0; len(q1)=16; i=1
Flow 1 status: 75a4bd6f-23a9-4ea0-a534-f6ab74e1556a: SUCCEEDED
len(q0)=0; len(q1)=15; i=2
Flow 2 status: e54363fa-10c3-4ba6-8623-9711ff39e8c4: SUCCEEDED
len(q0)=0; len(q1)

In [ ]:
print(flow_scope)

In [ ]:
for flow_action in q1:
    flows_client.flow_action_cancel(flow_id, flow_scope, flow_action['action_id'])

In [ ]:
#flows_client.flow_action_cancel(flow_id, flow_scope, '5e948f48-c3c3-4328-8d37-b5d1bc82df4c')

In [ ]:
#for ga_flow_action_id in ga_flow_actions.keys():
#    flows_client.flow_action_cancel(flow_id, flow_scope, ga_flow_action_id)

In [ ]:
#st = flows_client.flow_action_status(flow_id, flow_scope, '08925f04-569f-11e7-bef8-22000b9a448b')
#print(st['status'])

In [ ]:
!pwd